# Capítulo 7 — Remedios y métodos robustos

> **Overview**:
Compara OLS con errores estándar robustos (HC0–HC3), regresión robusta (Huber/Tukey) y bootstrap. Muestra que HC3 ensancha EE/IC de forma conservadora; RLM reduce influencia de outliers; bootstrap ofrece bandas empíricas adicionales.



**Definir ruta de datos**

In [1]:
from pathlib import Path

# Rutas deterministas: el capítulo se ejecuta desde la carpeta donde está este .md (p.ej. book/notebooks/)
DATA_PATH = Path("../data/AmesHousing_codificada.csv")  # relativo a book/notebooks/
BOOTSTRAP_OUT = Path("../data/bootstrap_df.csv")        # salida persistente en book/data/

assert DATA_PATH.is_file(), "No se encontró '../data/AmesHousing_codificada.csv'"
print("Usando CSV:", DATA_PATH.resolve())
print("Archivo bootstrap se guardará en:", BOOTSTRAP_OUT.resolve())

Usando CSV: /workspaces/ames-housing-project/book/data/AmesHousing_codificada.csv
Archivo bootstrap se guardará en: /workspaces/ames-housing-project/book/data/bootstrap_df.csv


## Correcciones para heterocedasticidad

De acuerdo con el supuesto de homocedasticidad ([Ecuación 6.2.1](#ecuacion-621-varianza-errores)), la presencia de heterocedasticidad puede provocar que los errores estándar de los coeficientes estén subestimados, afectando los valores $t$ y las decisiones de significancia. Para corregir este problema, se utilizan los estimadores de varianza robusta **HC** (Heteroscedasticity-Consistent), que ajustan los errores estándar sin cambiar los coeficientes estimados. 

- **HC0**: Estimador original de White, básico y consistente frente a heterocedasticidad.  
- **HC1**: Ajusta HC0 por los grados de libertad, corrigiendo ligeras subestimaciones.  
- **HC2**: Considera los *leverage points* de cada observación, dando más peso a observaciones influyentes.  
- **HC3**: Aproximación tipo *jackknife*, más conservadora y recomendada en muestras pequeñas por ofrecer errores estándar más cautelosos.

In [2]:
import statsmodels.api as sm
import pandas as pd

pd.set_option('display.float_format', '{:.6f}'.format)

data_modelo_base = pd.read_csv(DATA_PATH)

data_modelo_base = data_modelo_base[['SalePrice_log', 'Overall Qual', 'Gr Liv Area', 
                                     'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 
                                     'Full Bath', 'Year Built', 'Fireplaces', 'Lot Area']]
X = data_modelo_base[['Overall Qual', 'Gr Liv Area', 'Garage Cars', 'Total Bsmt SF',
                      '1st Flr SF', 'Full Bath', 'Year Built', 'Fireplaces', 'Lot Area']]
y = data_modelo_base[['SalePrice_log']]

X = sm.add_constant(X)

modelo_base = sm.OLS(y, X).fit()

resultados_HC0 = modelo_base.get_robustcov_results(cov_type='HC0')
resultados_HC1 = modelo_base.get_robustcov_results(cov_type='HC1')
resultados_HC2 = modelo_base.get_robustcov_results(cov_type='HC2')
resultados_HC3 = modelo_base.get_robustcov_results(cov_type='HC3')

def get_confint_df(result, var_names):
    ci = result.conf_int()
    if isinstance(ci, pd.DataFrame):
        ci = ci.loc[var_names]
    else:
        ci = pd.DataFrame(ci, columns=["lower", "upper"], index=var_names)
    return ci

variables = modelo_base.params.index

se_df = pd.DataFrame({
    "OLS": modelo_base.bse,
    "HC0": resultados_HC0.bse,
    "HC1": resultados_HC1.bse,
    "HC2": resultados_HC2.bse,
    "HC3": resultados_HC3.bse,
})
se_df.index.name = "Variable"

ic_ols = get_confint_df(modelo_base, variables)
ic_hc0 = get_confint_df(resultados_HC0, variables)
ic_hc1 = get_confint_df(resultados_HC1, variables)
ic_hc2 = get_confint_df(resultados_HC2, variables)
ic_hc3 = get_confint_df(resultados_HC3, variables)

ic_df = pd.DataFrame({
    "OLS_lower": ic_ols.iloc[:,0],
    "OLS_upper": ic_ols.iloc[:,1],
    "HC0_lower": ic_hc0.iloc[:,0],
    "HC0_upper": ic_hc0.iloc[:,1],
    "HC1_lower": ic_hc1.iloc[:,0],
    "HC1_upper": ic_hc1.iloc[:,1],
    "HC2_lower": ic_hc2.iloc[:,0],
    "HC2_upper": ic_hc2.iloc[:,1],
    "HC3_lower": ic_hc3.iloc[:,0],
    "HC3_upper": ic_hc3.iloc[:,1],
})
ic_df.index = variables
ic_df.index.name = "Variable"

display(se_df.style.format("{:.6f}"))

,OLS,HC0,HC1,HC2,HC3
Variable,,,,,
const,0.253443,0.289644,0.290168,0.290559,0.291480
Overall Qual,0.003334,0.004443,0.004451,0.004459,0.004474
Gr Liv Area,0.000010,0.000010,0.000010,0.000010,0.000011
Garage Cars,0.005301,0.006828,0.006841,0.006851,0.006874
Total Bsmt SF,0.000013,0.000015,0.000015,0.000015,0.000015
1st Flr SF,0.000014,0.000016,0.000016,0.000016,0.000016
Full Bath,0.007656,0.008385,0.008400,0.008412,0.008439
Year Built,0.000134,0.000151,0.000151,0.000151,0.000152
Fireplaces,0.005274,0.005640,0.005650,0.005658,0.005676


**Tabla 7.1.1.** Errores estándar OLS vs. HC0-HC3.

Se observa que los errores estándar aumentan ligeramente cuando se aplican las correcciones HC, especialmente para variables como `Overall Qual`, `Garage Cars` y `Full Bath`. Esto indica que los estimadores OLS originales podrían subestimar la variabilidad de los coeficientes si existe heterocedasticidad.

Por ejemplo, el coeficiente de `Overall Qual` tiene un error estándar de 0.00333 bajo OLS clásico, que se incrementa a 0.00447 bajo HC3, la corrección más conservadora. De manera similar, `Garage Cars` pasa de 0.00530 a 0.00687.  

Las variables con cambios mínimos en los errores estándar (como `Gr Liv Area` o `Lot Area`) sugieren que su variabilidad está poco afectada por heterocedasticidad.

In [3]:
display(ic_df.style.format("{:.6f}"))

,OLS_lower,OLS_upper,HC0_lower,HC0_upper,HC1_lower,HC1_upper,HC2_lower,HC2_upper,HC3_lower,HC3_upper
Variable,,,,,,,,,,
const,4.749133,5.743047,4.678149,5.814030,4.677121,5.815059,4.676355,5.815825,4.674550,5.817630
Overall Qual,0.095133,0.108206,0.092957,0.110382,0.092942,0.110397,0.092927,0.110412,0.092897,0.110442
Gr Liv Area,0.000220,0.000259,0.000219,0.000260,0.000219,0.000260,0.000219,0.000260,0.000219,0.000260
Garage Cars,0.038647,0.059434,0.035652,0.062430,0.035627,0.062454,0.035607,0.062475,0.035561,0.062520
Total Bsmt SF,0.000129,0.000178,0.000125,0.000182,0.000125,0.000182,0.000125,0.000182,0.000125,0.000183
1st Flr SF,-0.000026,0.000031,-0.000028,0.000033,-0.000028,0.000033,-0.000028,0.000033,-0.000028,0.000033
Full Bath,-0.028511,0.001512,-0.029940,0.002941,-0.029970,0.002971,-0.029993,0.002994,-0.030046,0.003047
Year Built,0.002500,0.003024,0.002467,0.003058,0.002466,0.003058,0.002466,0.003059,0.002465,0.003060
Fireplaces,0.035813,0.056495,0.035095,0.057213,0.035075,0.057233,0.035060,0.057248,0.035025,0.057283


**Tabla 7.1.2.** Intervalos de confianza OLS vs. HC0-HC3.

Se puede observar que los intervalos de confianza se ensanchan ligeramente cuando se aplican las correcciones HC, reflejando un aumento en la incertidumbre de los coeficientes. Por ejemplo, el coeficiente de `Overall Qual` tiene un intervalo de confianza de [0.095, 0.108] bajo OLS, que se amplía a [0.0929, 0.1104] con HC3. Lo mismo ocurre con `Garage Cars` y `Full Bath`, indicando que las inferencias de estos coeficientes son sensibles a la heterocedasticidad.

En contraste, intervalos de confianza prácticamente inalterados, como los de `Gr Liv Area` o `Lot Area`, sugieren que la heterocedasticidad tiene poco efecto sobre la precisión de estos coeficientes.  

En general, aplicar correcciones HC permite obtener intervalos de confianza más robustos, proporcionando inferencias más conservadoras y fiables cuando se sospecha heterocedasticidad.

## Modelos robustos con funciones Huber/Tukey

Los **modelos de regresión robusta** son una extensión de la regresión lineal ordinaria diseñada para reducir la influencia de outliers o valores atípicos en la estimación de los coeficientes. Mientras que la regresión OLS pondera todos los residuales por igual, los modelos robustos asignan **menor peso a los residuales grandes**, permitiendo obtener estimaciones más confiables y estables.

Existen diversas funciones de pérdida que determinan cómo se penalizan los residuales, donde cada una equilibra de distinta manera la eficiencia para valores centrales y la robustez frente a outliers extremos:

- **Huber / TukeyHuberT()**:

```{math}
:label: eq:7.2.1-huber
\rho(r) = 
\begin{cases} 
\frac{1}{2} r^2 & \text{si } |r| \le \delta \\[2mm]
\delta \left(|r| - \frac{1}{2}\delta \right) & \text{si } |r| > \delta
\end{cases}
```
  
_Ecuación_ {eq}`eq:7.2.1-huber`. Función de pérdida Huber.

Protege contra outliers moderados manteniendo eficiencia para valores centrales.

- **Tukey Biweight**:

```{math}
:label: eq:7.2.2-tukey
\rho(r) = 
\begin{cases}
\frac{c^2}{6} \left[ 1 - \left(1 - \left(\frac{r}{c}\right)^2 \right)^3 \right] & \text{si } |r| \le c \\[1mm]
\frac{c^2}{6} & \text{si } |r| > c
\end{cases}
```
  
_Ecuación_ {eq}`eq:7.2.2-tukey`. Función de pérdida Tukey.

Limita el impacto de observaciones lejanas sin eliminarlas completamente.

In [4]:
import statsmodels.api as sm
import pandas as pd

rlm_huber = sm.RLM(y, X, M=sm.robust.norms.HuberT()).fit()       # Función de pérdida Huber
rlm_tukey = sm.RLM(y, X, M=sm.robust.norms.TukeyBiweight()).fit() # Función de pérdida Tukey Biweight

rlm_df = pd.DataFrame({
    'OLS': modelo_base.params,
    'RLM_Huber': rlm_huber.params,
    'RLM_Tukey': rlm_tukey.params
})

weights_df = pd.DataFrame({
    'Huber_weights': rlm_huber.weights,
    'Tukey_weights': rlm_tukey.weights
})

display(rlm_df)

,OLS,RLM_Huber,RLM_Tukey
const,5.246090,5.554695,5.757333
Overall Qual,0.101670,0.093458,0.090582
Gr Liv Area,0.000240,0.000243,0.000245
Garage Cars,0.049041,0.055135,0.056782
Total Bsmt SF,0.000154,0.000169,0.000176
1st Flr SF,0.000003,-0.000008,-0.000013
Full Bath,-0.013499,-0.017119,-0.016548
Year Built,0.002762,0.002626,0.002528
Fireplaces,0.046154,0.040135,0.038912
Lot Area,0.000010,0.000011,0.000011


**Tabla 7.2.1.** Coeficientes OLS vs. RLM. *Valores en escala logarítmica.*

Se observa que el intercepto (`const`) aumenta al usar modelos robustos, lo que refleja que los outliers tienden a sesgar hacia abajo la estimación en OLS. Por su parte, variables como `Overall Qual` y `Fireplaces` muestran coeficientes algo menores en modelos robustos, indicando que su efecto estaba ligeramente sobreestimado por la presencia de outliers. 

Para la mayoría de las demás variables (`Gr Liv Area`, `Garage Cars`, `Total Bsmt SF`, `Year Built`), las diferencias son pequeñas, lo que sugiere que los outliers no tienen un impacto fuerte en estas estimaciones.

In [5]:
display(weights_df)

,Huber_weights,Tukey_weights
0,1.000000,0.953644
1,1.000000,0.866662
2,1.000000,0.970765
3,1.000000,0.922950
4,1.000000,0.962404
...,...,...
2763,1.000000,0.986576
2764,1.000000,0.999931
2765,1.000000,0.974192
2766,1.000000,0.993637


**Tabla 7.2.2.** Pesos outliers Huber vs. Tukey.

La mayoría de los pesos son cercanos a 1 en ambos modelos robustos, lo que indica que la mayoría de las observaciones se ajusta bien al modelo y tiene plena influencia en la estimación de los coeficientes. 

Sin embargo, algunas observaciones presentan pesos menores, como la última fila, con Huber = 0.69 y Tukey = 0.68, lo que refleja que su residuo es relativamente grande y su efecto en el ajuste se atenúa.

Además, es evidente que Tukey aplica un castigo más fuerte a residuales extremos.

## Bootstrap

El **bootstrap** es un método de remuestreo que permite estimar la variabilidad de los coeficientes de un modelo sin asumir una distribución específica de los errores. Consiste en generar múltiples muestras con reemplazo a partir de los datos originales y recalcular los estimadores para cada réplica, obteniendo así una **distribución empírica** de los coeficientes, a partir de la cual se calculan el error estándar y los intervalos de confianza.

In [6]:
from sklearn.utils import resample
import numpy as np
import pandas as pd

B = 1000
coef_boot = np.zeros((B, X.shape[1]))

for i in range(B):
    X_resample, y_resample = resample(X, y)
    model_bs = sm.OLS(y_resample, X_resample).fit()
    coef_boot[i, :] = model_bs.params

coef_mean = coef_boot.mean(axis=0)        
coef_se = coef_boot.std(axis=0)                     
ic_lower = np.percentile(coef_boot, 2.5, axis=0)       
ic_upper = np.percentile(coef_boot, 97.5, axis=0)  

bootstrap_df = pd.DataFrame({
    'Coef_mean': coef_mean,
    'SE_bootstrap': coef_se,
    'IC_2.5%': ic_lower,
    'IC_97.5%': ic_upper
}, index=X.columns)

bootstrap_df.to_csv('bootstrap_df.csv', sep=",", index=False)

bootstrap_df

,Coef_mean,SE_bootstrap,IC_2.5%,IC_97.5%
const,5.246026,0.293715,4.690256,5.817355
Overall Qual,0.101666,0.004578,0.093013,0.111151
Gr Liv Area,0.000240,0.000011,0.000219,0.000261
Garage Cars,0.049096,0.006866,0.035724,0.061879
Total Bsmt SF,0.000153,0.000015,0.000124,0.000184
1st Flr SF,0.000003,0.000016,-0.000028,0.000035
Full Bath,-0.013517,0.008340,-0.030215,0.001709
Year Built,0.002762,0.000153,0.002466,0.003054
Fireplaces,0.045680,0.005796,0.034195,0.056856
Lot Area,0.000010,0.000001,0.000008,0.000012


**Tabla 7.3.1 — Resumen de bootstrap de coeficientes.** Esta tabla resume los resultados producidos por el código anterior.
Discusión: compare magnitudes relativas. En presencia de heterocedasticidad,
los SE HC3 suelen ser mayores que OLS. Si los IC se ensanchan (ver Tabla 7.2), las
conclusiones sobre significancia pueden cambiar. Con bootstrap, valide la estabilidad
de los coeficientes frente a remuestreo. (Ver «Takeaways» al final.)

Se observa que variables como `Overall Qual`, `Gr Liv Area`, `Garage Cars` y `Year Built` tienen coeficientes claramente distintos de cero, con intervalos de confianza estrechos y consistentes, lo que sugiere estimaciones robustas y estables. Por el contrario, `1st Flr SF` y `Full Bath` presentan intervalos que incluyen el cero, indicando que su efecto sobre la variable respuesta podría no ser significativo.

## OLS vs. HC3 vs. Bootstrap

In [7]:
import pandas as pd
# 1Extraer y alinear objetos base
coef_ols = modelo_base.params
se_ols = modelo_base.bse
se_hc3 = resultados_HC3.bse

# Asegurar que bootstrap_df esté indexado por el nombre del coeficiente (si aplica)
if isinstance(bootstrap_df, pd.DataFrame) and 'variable' in bootstrap_df.columns and bootstrap_df.index.name != 'variable':
    bootstrap_df = bootstrap_df.set_index('variable')

coef_boot_mean = bootstrap_df['Coef_mean'].reindex(coef_ols.index)
se_boot = bootstrap_df['SE_bootstrap'].reindex(coef_ols.index)

# 2 Anchuras de IC (97.5 - 2.5) para cada método
ic_ols_width = (modelo_base.conf_int().iloc[:, 1] - modelo_base.conf_int().iloc[:, 0]).reindex(coef_ols.index)

hc3_ci = resultados_HC3.conf_int()
# Normalizar a DataFrame con el mismo índice
if isinstance(hc3_ci, pd.DataFrame):
    hc3_ci_df = hc3_ci
else:
    hc3_ci_df = pd.DataFrame(hc3_ci, index=coef_ols.index, columns=[0, 1])

ic_hc3_width = (hc3_ci_df.iloc[:, 1] - hc3_ci_df.iloc[:, 0]).reindex(coef_ols.index)
ic_boot_width = (bootstrap_df['IC_97.5%'] - bootstrap_df['IC_2.5%']).reindex(coef_ols.index)

# 3 Tabla maestra comparativa
comparative_df = pd.DataFrame({
    'Coef_OLS': coef_ols,
    'Coef_Bootstrap': coef_boot_mean,
    'SE_OLS': se_ols,
    'SE_HC3': se_hc3,
    'SE_Bootstrap': se_boot,
    'IC_width_OLS': ic_ols_width,
    'IC_width_HC3': ic_hc3_width,
    'IC_width_Bootstrap': ic_boot_width
})

# 4 Subtablas
coef_df = comparative_df[['Coef_OLS', 'Coef_Bootstrap']]
se_df = comparative_df[['SE_OLS', 'SE_HC3', 'SE_Bootstrap']]
ic_df = comparative_df[['IC_width_OLS', 'IC_width_HC3', 'IC_width_Bootstrap']]

# Mostrar por conveniencia
coef_df

,Coef_OLS,Coef_Bootstrap
const,5.246090,5.246026
Overall Qual,0.101670,0.101666
Gr Liv Area,0.000240,0.000240
Garage Cars,0.049041,0.049096
Total Bsmt SF,0.000154,0.000153
1st Flr SF,0.000003,0.000003
Full Bath,-0.013499,-0.013517
Year Built,0.002762,0.002762
Fireplaces,0.046154,0.045680
Lot Area,0.000010,0.000010


**Tabla 7.4.1— Coeficientes OLS vs. Bootstrap.** *Valores en escala logarítmica.* Esta tabla resume los resultados producidos por el código anterior.
Discusión: compare magnitudes relativas. En presencia de heterocedasticidad,
los SE HC3 suelen ser mayores que OLS. Si los IC se ensanchan (ver Tabla 7.2), las
conclusiones sobre significancia pueden cambiar. Con bootstrap, valide la estabilidad
de los coeficientes frente a remuestreo. (Ver «Takeaways» al final.)

Se observa que la estimación de los parámetros es muy estable frente al remuestreo, lo que sugiere que la muestra utilizada es suficientemente representativa y que los coeficientes no dependen excesivamente de observaciones individuales.

En particular, las variables como `Overall Qual`, `Gr Liv Area` y `Fireplaces` muestran coeficientes positivos consistentes en ambos métodos, confirmando su relación directa con el precio de la vivienda. Por su parte, `Full Bath` mantiene un coeficiente ligeramente negativo, indicando que, controlando por las demás variables, su efecto es mínimo.

In [8]:
se_df

,SE_OLS,SE_HC3,SE_Bootstrap
const,0.253443,0.291480,0.293715
Overall Qual,0.003334,0.004474,0.004578
Gr Liv Area,0.000010,0.000011,0.000011
Garage Cars,0.005301,0.006874,0.006866
Total Bsmt SF,0.000013,0.000015,0.000015
1st Flr SF,0.000014,0.000016,0.000016
Full Bath,0.007656,0.008439,0.008340
Year Built,0.000134,0.000152,0.000153
Fireplaces,0.005274,0.005676,0.005796
Lot Area,0.000001,0.000001,0.000001


**Tabla 7.4.2 — Errores estándar comparados (OLS vs. HC0–HC3 vs. Bootstrap).** Esta tabla resume los resultados producidos por el código anterior.
Discusión: compare magnitudes relativas. En presencia de heterocedasticidad,
los SE HC3 suelen ser mayores que OLS. Si los IC se ensanchan (ver Tabla 7.2), las
conclusiones sobre significancia pueden cambiar. Con bootstrap, valide la estabilidad
de los coeficientes frente a remuestreo. (Ver «Takeaways» al final.)


Los errores estándar de OLS son generalmente menores que los obtenidos mediante HC3 o bootstrap, lo que sugiere que este modelo inicial podría subestimar la incertidumbre cuando hay heterocedasticidad presente o dependiendo de la muestra seleccionada.  

El método HC3, diseñado para ser más conservador frente a heterocedasticidad y leverage points, produce errores estándar ligeramente mayores que OLS, especialmente para variables como `Overall Qual`, `Garage Cars` y `Full Bath`. El bootstrap refleja un patrón muy similar al de HC3, confirmando la estabilidad de las estimaciones.

Es notable que las variables con errores estándar relativamente bajos (`Gr Liv Area`, `Lot Area`) están estimadas con gran precisión, mientras que aquellas con errores más altos (`Full Bath`, `Garage Cars`) presentan mayor incertidumbre en la estimación de su efecto sobre el precio de la vivienda.

In [9]:
ic_df

,IC_width_OLS,IC_width_HC3,IC_width_Bootstrap
const,0.993914,1.143080,1.127099
Overall Qual,0.013073,0.017545,0.018138
Gr Liv Area,0.000039,0.000041,0.000042
Garage Cars,0.020787,0.026959,0.026155
Total Bsmt SF,0.000049,0.000058,0.000060
1st Flr SF,0.000057,0.000062,0.000064
Full Bath,0.030023,0.033094,0.031925
Year Built,0.000524,0.000595,0.000587
Fireplaces,0.020682,0.022258,0.022661
Lot Area,0.000003,0.000004,0.000004


**Tabla 7.4.3 — Ancho de intervalos de confianza por método.** Esta tabla resume los resultados producidos por el código anterior.
Discusión: compare magnitudes relativas. En presencia de heterocedasticidad,
los SE HC3 suelen ser mayores que OLS. Si los IC se ensanchan (ver Tabla 7.2), las
conclusiones sobre significancia pueden cambiar. Con bootstrap, valide la estabilidad
de los coeficientes frente a remuestreo. (Ver «Takeaways» al final.)

Alineados con los errores estándar ([Tabla 7.4.2](#tabla-742-se-ols-hc3-boot)), los intervalos calculados con OLS son consistentemente más estrechos que los obtenidos con HC3 o bootstrap, llegando a la misma conclusión de que el OLS inicial es menos robusto.

> **Key takeaways**
>- **Sobre heterocedasticidad.** Si los errores estándar HC3 (o HC2) superan de manera consistente a los de OLS, esto sugiere varianza no constante. En tal caso, la inferencia debe basarse en versiones robustas (HC3 recomendado).
Además, el **ancho de los IC** es un buen indicador del impacto en la precisión de la estimación.
>- **Sobre RLM (Huber/Tukey).** Cuando existen outliers influyentes, RLM puede reducir su peso. Cambios sustanciales en coeficientes o en su significancia, frente a OLS, ameritan diagnosticar
casos influyentes y revisar la especificación.
>- **Sobre bootstrap.** El bootstrap  brinda una validación empírica de la variabilidad de los parámetros.